In [0]:
from pyspark.sql.functions import lit ,max,min

# **📊 1- What are all available starting classes and their base stats?**

**Get a complete overview of all starting options to understand the foundation of each class.**

In [0]:
%sql
SELECT * FROM eldenringcatalog.gold.startingclasses 
ORDER BY Level DESC

Class_Name,Level,Vigor,Mind,Endurance,Strength,Dexterity,Intelligence,Faith,Arcane
Confessor,10,10,13,10,12,12,9,14,9
Samurai,9,12,11,13,12,15,9,8,8
Prisoner,9,11,12,11,11,14,14,6,9
Vagabond,9,15,10,11,14,13,9,9,7
Warrior,8,11,12,11,10,16,10,8,9
Prophet,7,10,14,8,11,10,7,16,10
Hero,7,14,9,12,16,9,7,8,11
Astrologer,6,9,15,9,8,12,16,7,9
Bandit,5,10,11,10,9,13,9,8,14
Wretch,1,10,10,10,10,10,10,10,10


Databricks visualization. Run in Databricks to view.

# **📊 2- Which class begins at the highest starting level, and what is the total of all stats for each class?**

**Higher starting level means more total stat points allocated, which could provide advantages.**

In [0]:
from pyspark.sql import functions as F

# Columns to sum (exclude Class_Name and Level)
cols_to_sum = [
    c for c, t in df.dtypes
    if t in ("int", "bigint") and c not in ("Class_Name", "Level")
]

# Aggregate dataframe
df_sum_by_class = (
    df
    .groupBy("Class_Name")
    .agg(
        F.first("Level").alias("Level"),           # keep Level without summing
        *[F.sum(c).alias(c) for c in cols_to_sum], 
        F.sum(sum(F.col(c) for c in cols_to_sum)).alias("total_sum")
    )
    .orderBy(F.col("total_sum").desc())
)

display(df_sum_by_class)


Class_Name,Level,Vigor,Mind,Endurance,Strength,Dexterity,Intelligence,Faith,Arcane,total_sum
Confessor,10,10,13,10,12,12,9,14,9,89
Samurai,9,12,11,13,12,15,9,8,8,88
Prisoner,9,11,12,11,11,14,14,6,9,88
Vagabond,9,15,10,11,14,13,9,9,7,88
Warrior,8,11,12,11,10,16,10,8,9,87
Prophet,7,10,14,8,11,10,7,16,10,86
Hero,7,14,9,12,16,9,7,8,11,86
Astrologer,6,9,15,9,8,12,16,7,9,85
Bandit,5,10,11,10,9,13,9,8,14,84
Wretch,1,10,10,10,10,10,10,10,10,80


Databricks visualization. Run in Databricks to view.

**The **Wretch** class starts at the lowest level, Level 1, with a total sum of all other attributes (Vigor, Mind, Endurance, Strength, Dexterity, Intelligence, Faith, Arcane) equal to 80. In contrast, the **Confesso**r class starts at the highest level, Level 10, with a total sum of attributes equal to 89**

# **📊 4 - Which class has the most balanced stat distribution?**

**A balanced class is versatile for experimentation and hybrid builds**

In [0]:
%sql
WITH class_stats AS (
    SELECT 
        Class_Name,
        (Vigor + Mind + Endurance + Strength + Dexterity + Intelligence + Faith + Arcane) / 8.0 AS mean_stat,
        Vigor, Mind, Endurance, Strength, Dexterity, Intelligence, Faith, Arcane
    FROM eldenringcatalog.gold.startingclasses
)
SELECT
    Class_Name,
    mean_stat,
    SQRT(
        (POW(Vigor - mean_stat, 2) +
         POW(Mind - mean_stat, 2) +
         POW(Endurance - mean_stat, 2) +
         POW(Strength - mean_stat, 2) +
         POW(Dexterity - mean_stat, 2) +
         POW(Intelligence - mean_stat, 2) +
         POW(Faith - mean_stat, 2) +
         POW(Arcane - mean_stat, 2)) / 8.0
    ) AS std_dev
FROM class_stats
ORDER BY std_dev;


Class_Name,mean_stat,std_dev
Wretch,10.000000,0.0
Confessor,11.125000,1.7633419974582356
Bandit,10.500000,1.9364916731037085
Warrior,10.875000,2.2603926650031405
Samurai,11.000000,2.345207879911715
Prisoner,11.000000,2.449489742783178
Vagabond,11.000000,2.598076211353316
Prophet,10.750000,2.7726341266023544
Hero,10.750000,2.9047375096555625
Astrologer,10.625000,3.119995993587171


**Wretch is the most balanced class with all stats at 10. Lower standard deviation indicates more balanced distribution.**

# **📊 5- Which class is best for a pure Strength build**

**Identify the optimal starting class for players focusing on Strength weapons**

In [0]:
%sql
SELECT 
    Class_Name,
    Level,
    Strength,
    (Strength + Vigor + Endurance) AS Primary_Stats,
    (Intelligence + Faith + Arcane + CASE WHEN Mind > 10 THEN Mind - 10 ELSE 0 END) AS Wasted_Stats
FROM eldenringcatalog.gold.startingclasses
ORDER BY Strength DESC, Wasted_Stats;

Class_Name,Level,Strength,Primary_Stats,Wasted_Stats
Hero,7,16,42,26
Vagabond,9,14,40,25
Samurai,9,12,37,26
Confessor,10,12,32,35
Prisoner,9,11,33,31
Prophet,7,11,29,37
Warrior,8,10,32,29
Wretch,1,10,30,30
Bandit,5,9,29,32
Astrologer,6,8,26,37


**Hero has the highest Strength (16) and relatively low investment in magic stats, making it ideal for pure Strength builds.**

# **📊 6 - Which class is best for a pure Dexterity build?**

**Identify the optimal starting class for Dexterity-focused builds.**

In [0]:
%sql
SELECT 
    Class_Name,
    Level,
    Dexterity,
    (Dexterity + Vigor + Endurance) AS Primary_Stats,
    (Intelligence + Faith + Arcane + CASE WHEN Mind > 10 THEN Mind - 10 ELSE 0 END) AS Wasted_Stats
FROM eldenringcatalog.gold.startingclasses
ORDER BY Dexterity DESC, Wasted_Stats;

Class_Name,Level,Dexterity,Primary_Stats,Wasted_Stats
Warrior,8,16,38,29
Samurai,9,15,40,26
Prisoner,9,14,36,31
Vagabond,9,13,39,25
Bandit,5,13,33,32
Confessor,10,12,32,35
Astrologer,6,12,30,37
Wretch,1,10,30,30
Prophet,7,10,28,37
Hero,7,9,35,26


**Warrior has the highest Dexterity (16), making it the premier choice for Dexterity builds.**


# **📊 7 - Which class is best for an Intelligence (Sorcery) build?**

**Mages need high Intelligence and Mind for spells and FP.**

In [0]:
%sql
SELECT 
    Class_Name,
    Level,
    Intelligence,
    Mind,
    (Intelligence + Mind + Vigor) AS Primary_Stats,
    (Strength + Dexterity + Faith + Arcane - 30) AS Wasted_Stats --# Allow some for weapon requirements
FROM eldenringcatalog.gold.startingclasses
ORDER BY Intelligence DESC, Wasted_Stats;

Class_Name,Level,Intelligence,Mind,Primary_Stats,Wasted_Stats
Astrologer,6,16,15,40,6
Prisoner,9,14,12,37,10
Wretch,1,10,10,30,10
Warrior,8,10,12,33,13
Vagabond,9,9,10,34,13
Samurai,9,9,11,32,13
Bandit,5,9,11,30,14
Confessor,10,9,13,32,17
Hero,7,7,9,30,14
Prophet,7,7,14,31,17


**Astrologer has the highest Intelligence (16) and good Mind (15), perfect for sorcery builds.**

# **📊 Question 8- Which class is best for a Faith (Incantation) build?**

**Faith builds focus on incantations and require high Faith and Mind**

In [0]:
%sql
SELECT 
    Class_Name,
    Level,
    Faith,
    Mind,
    (Faith + Mind + Vigor) AS Primary_Stats,
    (Strength + Dexterity + Intelligence + Arcane - 30) AS Wasted_Stats
FROM eldenringcatalog.gold.startingclasses
ORDER BY Faith DESC, Wasted_Stats;

Class_Name,Level,Faith,Mind,Primary_Stats,Wasted_Stats
Prophet,7,16,14,40,8
Confessor,10,14,13,37,12
Wretch,1,10,10,30,10
Vagabond,9,9,10,34,13
Hero,7,8,9,31,13
Samurai,9,8,11,31,14
Warrior,8,8,12,31,15
Bandit,5,8,11,29,15
Astrologer,6,7,15,31,15
Prisoner,9,6,12,29,18


**Prophet has the highest Faith (16) with good Mind (14), ideal for faith-based builds.**

# **📊 9 - Which class is best for an Arcane build?**

**Arcane builds focus on status effects and special weapons/spells**

In [0]:
%sql
SELECT 
    Class_Name,
    Level,
    Arcane,
    Dexterity,
    (Arcane + Vigor + Dexterity) AS Primary_Stats,
    (Intelligence + Faith + CASE WHEN Strength > 10 THEN Strength - 10 ELSE 0 END) AS Wasted_Stats
FROM eldenringcatalog.gold.startingclasses
ORDER BY Arcane DESC, Wasted_Stats;

Class_Name,Level,Arcane,Dexterity,Primary_Stats,Wasted_Stats
Bandit,5,14,13,37,17
Hero,7,11,9,34,21
Wretch,1,10,10,30,20
Prophet,7,10,10,30,24
Warrior,8,9,16,36,18
Prisoner,9,9,14,34,21
Astrologer,6,9,12,30,23
Confessor,10,9,12,31,25
Samurai,9,8,15,35,19
Vagabond,9,7,13,35,22


**Bandit has the highest Arcane (14), making it best for bleed and arcane-scaling builds.**

# **10- What is the minimum stat value across all classes and attributes?**

**Understand the floor values for stat allocation.**

In [0]:
df_min_stats = df_classes.agg(
    min("Vigor").alias("Min_Vigor"),
    min("Mind").alias("Min_Mind"),
    min("Endurance").alias("Min_Endurance"),
    min("Strength").alias("Min_Strength"),
    min("Dexterity").alias("Min_Dexterity"),
    min("Intelligence").alias("Min_Intelligence"),
    min("Faith").alias("Min_Faith"),
    min("Arcane").alias("Min_Arcane")
)

display(df_min_stats)

stats_list = ["Vigor", "Mind", "Endurance", "Strength", "Dexterity", "Intelligence", "Faith", "Arcane"]

for stat in stats_list:
    min_val = df_classes.agg(min(stat)).collect()[0][0]
    classes_with_min = df_classes.filter(col(stat) == min_val).select("Class_Name", stat)
    print(f"\n{stat} minimum ({min_val}):")
    classes_with_min.show(truncate=False)

Min_Vigor,Min_Mind,Min_Endurance,Min_Strength,Min_Dexterity,Min_Intelligence,Min_Faith,Min_Arcane
9,9,8,8,9,7,6,7



Vigor minimum (9):
+----------+-----+
|Class_Name|Vigor|
+----------+-----+
|Astrologer|9    |
+----------+-----+


Mind minimum (9):
+----------+----+
|Class_Name|Mind|
+----------+----+
|Hero      |9   |
+----------+----+


Endurance minimum (8):
+----------+---------+
|Class_Name|Endurance|
+----------+---------+
|Prophet   |8        |
+----------+---------+


Strength minimum (8):
+----------+--------+
|Class_Name|Strength|
+----------+--------+
|Astrologer|8       |
+----------+--------+


Dexterity minimum (9):
+----------+---------+
|Class_Name|Dexterity|
+----------+---------+
|Hero      |9        |
+----------+---------+


Intelligence minimum (7):
+----------+------------+
|Class_Name|Intelligence|
+----------+------------+
|Hero      |7           |
|Prophet   |7           |
+----------+------------+


Faith minimum (6):
+----------+-----+
|Class_Name|Faith|
+----------+-----+
|Prisoner  |6    |
+----------+-----+


Arcane minimum (7):
+----------+------+
|Class_Name|Arcane|

**Most stats have a minimum of 7-9 across all classes. Prisoner has the lowest Faith (6).**

# **11 - What is the maximum stat value across all classes and attributes?**

**Identify the ceiling for starting stats to understand specialization.**

In [0]:
df_max_stats = df_classes.agg(
    max(col("Vigor")).alias("Max_Vigor"),
    max(col("Mind")).alias("Max_Mind"),
    max(col("Endurance")).alias("Max_Endurance"),
    max(col("Strength")).alias("Max_Strength"),
    max(col("Dexterity")).alias("Max_Dexterity"),
    max(col("Intelligence")).alias("Max_Intelligence"),
    max(col("Faith")).alias("Max_Faith"),
    max(col("Arcane")).alias("Max_Arcane")
)

display(df_max_stats)

# Show which class has the maximum for each stat
for stat in stats_list:
    max_val = df_classes.agg(max(stat)).collect()[0][0]
    classes_with_max = df_classes.filter(col(stat) == max_val).select("Class_Name", stat)
    print(f"\n{stat} maximum ({max_val}):")
    classes_with_max.show(truncate=False)

Max_Vigor,Max_Mind,Max_Endurance,Max_Strength,Max_Dexterity,Max_Intelligence,Max_Faith,Max_Arcane
15,15,13,16,16,16,16,14



Vigor maximum (15):
+----------+-----+
|Class_Name|Vigor|
+----------+-----+
|Vagabond  |15   |
+----------+-----+


Mind maximum (15):
+----------+----+
|Class_Name|Mind|
+----------+----+
|Astrologer|15  |
+----------+----+


Endurance maximum (13):
+----------+---------+
|Class_Name|Endurance|
+----------+---------+
|Samurai   |13       |
+----------+---------+


Strength maximum (16):
+----------+--------+
|Class_Name|Strength|
+----------+--------+
|Hero      |16      |
+----------+--------+


Dexterity maximum (16):
+----------+---------+
|Class_Name|Dexterity|
+----------+---------+
|Warrior   |16       |
+----------+---------+


Intelligence maximum (16):
+----------+------------+
|Class_Name|Intelligence|
+----------+------------+
|Astrologer|16          |
+----------+------------+


Faith maximum (16):
+----------+-----+
|Class_Name|Faith|
+----------+-----+
|Prophet   |16   |
+----------+-----+


Arcane maximum (14):
+----------+------+
|Class_Name|Arcane|
+----------+-----

**Maximum starting values are typically 14-16 for specialized stats. Warrior, Hero, Astrologer, and Prophet each excel in their primary attributes.**


# **📊 12- Which classes have at least 12 in both Strength and Dexterity (quality builds)?**

**Quality builds use weapons that scale with both STR and DEX.**

In [0]:
%sql
SELECT 
    Class_Name,
    Level,
    Strength,
    Dexterity,
    (Strength + Dexterity) AS Quality_Stats,
    Vigor,
    Endurance
FROM eldenringcatalog.gold.startingclasses
WHERE Strength >= 12 AND Dexterity >= 12
ORDER BY Quality_Stats DESC;

Class_Name,Level,Strength,Dexterity,Quality_Stats,Vigor,Endurance
Vagabond,9,14,13,27,15,11
Samurai,9,12,15,27,12,13
Confessor,10,12,12,24,10,10


**Vagabond, Samurai, and Confessor are good starting points for quality builds with balanced STR/DEX.**

# **📊 13- Which classes are best for hybrid INT/FAI builds (both magic types)?**

In [0]:
%sql
SELECT 
    Class_Name,
    Level,
    Intelligence,
    Faith,
    Mind,
    (Intelligence + Faith) AS Total_Magic_Stats,
    (Strength + Dexterity) AS Physical_Stats,
    CASE 
        WHEN Intelligence > Faith THEN Faith * 1.0 / Intelligence
        ELSE Intelligence * 1.0 / Faith
    END AS Magic_Balance
FROM eldenringcatalog.gold.startingclasses
WHERE Intelligence >= 9 AND Faith >= 9
ORDER BY Magic_Balance DESC, Total_Magic_Stats DESC;

Class_Name,Level,Intelligence,Faith,Mind,Total_Magic_Stats,Physical_Stats,Magic_Balance
Wretch,1,10,10,10,20,20,1.000000000000
Vagabond,9,9,9,10,18,27,1.000000000000
Confessor,10,9,14,13,23,24,0.642857142857


**Confessor and Wretch provide the strongest starting points for hybrid Intelligence/Faith builds.
The Wretch has a total of 20 magic-related stats and achieves a perfect magic balance score of 1.
The Confessor has a higher total magic stat value, but its magic balance ratio is lower at 0.64.**

# **📊 14 - Calculate the "waste factor" for each class relative to specific builds**

**When choosing a class, you want to minimize stats you'll never use.**

In [0]:
%sql
SELECT 
    Class_Name,
    Level,
    (Intelligence + Faith + Arcane + CASE WHEN Mind > 12 THEN Mind - 12 ELSE 0 END) AS STR_Build_Waste,
    (Intelligence + Faith + Arcane + CASE WHEN Mind > 12 THEN Mind - 12 ELSE 0 END) AS DEX_Build_Waste,
    (Strength + Faith + Arcane + CASE WHEN Dexterity > 12 THEN Dexterity - 12 ELSE 0 END) AS INT_Build_Waste,
    (Strength + Intelligence + Arcane + CASE WHEN Dexterity > 12 THEN Dexterity - 12 ELSE 0 END) AS FAI_Build_Waste
FROM eldenringcatalog.gold.startingclasses
ORDER BY STR_Build_Waste;

Class_Name,Level,STR_Build_Waste,DEX_Build_Waste,INT_Build_Waste,FAI_Build_Waste
Samurai,9,25,25,31,32
Vagabond,9,25,25,31,31
Hero,7,26,26,35,34
Warrior,8,27,27,31,33
Prisoner,9,29,29,28,36
Wretch,1,30,30,30,30
Bandit,5,31,31,32,33
Confessor,10,33,33,35,30
Astrologer,6,35,35,24,33
Prophet,7,35,35,37,28


**Each specialized class (Hero, Warrior, Astrologer, Prophet) minimizes waste for their respective build types.**

# **📊 15 - Which class requires the LEAST additional leveling to reach 40 Vigor?**


**Vigor is crucial for survivability. Calculate who gets to 40 Vigor fastest**

In [0]:
%sql
SELECT 
    Class_Name,
    Level,
    Vigor,
    (40 - Vigor) AS Levels_To_40_Vigor,
    (Level + (40 - Vigor)) AS Final_Level
FROM eldenringcatalog.gold.startingclasses
ORDER BY Levels_To_40_Vigor;

Class_Name,Level,Vigor,Levels_To_40_Vigor,Final_Level
Vagabond,9,15,25,34
Hero,7,14,26,33
Samurai,9,12,28,37
Prisoner,9,11,29,38
Warrior,8,11,29,37
Prophet,7,10,30,37
Confessor,10,10,30,40
Bandit,5,10,30,35
Wretch,1,10,30,31
Astrologer,6,9,31,37


**Vagabond starts with 15 Vigor, requiring only 25 levels to reach 40, the fastest path to high HP**

# **📊 16- Compare average physical stats (STR+DEX) vs magic stats (INT+FAI) per class**

**Understand whether each class leans physical or magical.**

In [0]:
%sql
SELECT 
    Class_Name,
    (Strength + Dexterity) AS Physical_Stats,
    (Intelligence + Faith) AS Magic_Stats,
    ((Strength + Dexterity) - (Intelligence + Faith)) AS Physical_Advantage,
    CASE 
        WHEN (Strength + Dexterity) > (Intelligence + Faith) THEN 'Physical'
        WHEN (Strength + Dexterity) < (Intelligence + Faith) THEN 'Magic'
        ELSE 'Hybrid'
    END AS Build_Type
FROM eldenringcatalog.gold.startingclasses
ORDER BY Physical_Advantage DESC;


Class_Name,Physical_Stats,Magic_Stats,Physical_Advantage,Build_Type
Hero,25,15,10,Physical
Samurai,27,17,10,Physical
Vagabond,27,18,9,Physical
Warrior,26,18,8,Physical
Bandit,22,17,5,Physical
Prisoner,25,20,5,Physical
Confessor,24,23,1,Physical
Wretch,20,20,0,Hybrid
Prophet,21,23,-2,Magic
Astrologer,20,23,-3,Magic


In [0]:
%sql
SELECT 
    CASE 
        WHEN (Strength + Dexterity) > (Intelligence + Faith) THEN 'Physical'
        WHEN (Strength + Dexterity) < (Intelligence + Faith) THEN 'Magic'
        ELSE 'Hybrid'
    END AS Build_Type,
    COUNT(*) as count
FROM eldenringcatalog.gold.startingclasses
GROUP BY Build_Type;

Build_Type,count
Physical,7
Magic,2
Hybrid,1


**Our lineup includes seven classes focused on physical combat, two focused on magic, and one balanced hybrid class.**

# **17 - Which class has the highest Mind stat (most starting FP)?**


**Mind determines FP pool for spells and weapon arts**

In [0]:
%sql
SELECT 
    Class_Name,
    Level,
    Mind,
    Intelligence,
    Faith,
    (Intelligence + Faith) AS Magic_Stats,
    ROUND(Mind * 1.0 / ((Intelligence + Faith) + 1), 2) AS Mind_To_Magic_Ratio
FROM eldenringcatalog.gold.startingclasses
ORDER BY Mind DESC;

Class_Name,Level,Mind,Intelligence,Faith,Magic_Stats,Mind_To_Magic_Ratio
Astrologer,6,15,16,7,23,0.63
Prophet,7,14,7,16,23,0.58
Confessor,10,13,9,14,23,0.54
Prisoner,9,12,14,6,20,0.57
Warrior,8,12,10,8,18,0.63
Samurai,9,11,9,8,17,0.61
Bandit,5,11,9,8,17,0.61
Vagabond,9,10,9,9,18,0.53
Wretch,1,10,10,10,20,0.48
Hero,7,9,7,8,15,0.56


**Astrologer has the highest Mind (15), providing the largest FP pool for spell-heavy builds.**

# **📊18 - Create a "versatility score" for each class**

## **Some players want flexibility to try different builds without respeccing.**

In [0]:
%sql
SELECT 
    Class_Name,
    Level,
    LEAST(Vigor, Mind, Endurance, Strength, Dexterity, Intelligence, Faith, Arcane) AS min_stat,
    GREATEST(Vigor, Mind, Endurance, Strength, Dexterity, Intelligence, Faith, Arcane) AS max_stat,
    (GREATEST(Vigor, Mind, Endurance, Strength, Dexterity, Intelligence, Faith, Arcane) - 
     LEAST(Vigor, Mind, Endurance, Strength, Dexterity, Intelligence, Faith, Arcane)) AS stat_range,
    ROUND(100.0 / ((GREATEST(Vigor, Mind, Endurance, Strength, Dexterity, Intelligence, Faith, Arcane) - 
                    LEAST(Vigor, Mind, Endurance, Strength, Dexterity, Intelligence, Faith, Arcane)) + 1), 2) AS versatility_score
FROM eldenringcatalog.gold.startingclasses
ORDER BY versatility_score DESC;

Class_Name,Level,min_stat,max_stat,stat_range,versatility_score
Wretch,1,10,10,0,100.00
Confessor,10,9,14,5,16.67
Bandit,5,8,14,6,14.29
Samurai,9,8,15,7,12.50
Warrior,8,8,16,8,11.11
Vagabond,9,7,15,8,11.11
Prisoner,9,6,14,8,11.11
Astrologer,6,7,16,9,10.00
Hero,7,7,16,9,10.00
Prophet,7,7,16,9,10.00


**Wretch is the most versatile class with all stats at 10. Confessor and Bandit also offer good versatility with balanced distributions.**

# **Summary of Key Findings and Recommendations**

### **🎯 Key Findings**

**1. Class Level Distribution**

* Highest Starting Level: Confessor (Level 10, 89 total stats)
* Lowest Starting Level: Wretch (Level 1, 80 total stats)
* Higher starting levels provide more allocated stat points but may include "wasted" stats for specialized builds

**2. Most Balanced Class**

* Wretch has the most balanced stat distribution (all stats at 10)
* Lowest standard deviation across all attributes
* deal for players who want maximum flexibility and experimentation

**3. Best Classes by Build Type**

Physical Builds:

  * Strength Build: Hero (16 STR, minimal magic stat waste)
  * Dexterity Build: Warrior (16 DEX, optimal for DEX scaling weapons)
  * Quality Build (STR+DEX): Vagabond, Samurai, and Confessor (balanced 12+ in both stats)

Magic Builds:

  * Intelligence/Sorcery: Astrologer (16 INT, 15 Mind - highest FP pool)
  * Faith/Incantations: Prophet (16 Faith, 14 Mind)
  * Hybrid INT/FAI: Confessor (best total magic stats) or Wretch (perfect 1.0 magic balance)

Specialized:

  * Arcane Build: Bandit (14 Arcane, optimal for bleed/status effect builds)

**4. Survivability Analysis**

* Fastest to 40 Vigor: Vagabond (starts at 15 Vigor, needs only 25 levels)
* Critical for early-game survival and endgame viability
* Highest Mind: Astrologer (15 Mind = largest FP pool for spell casting)

**5. Stat Optimizatio**n

* Minimum Stats: Range from 6-9 (Prisoner has lowest Faith at 6)
* Maximum Stats: Range from 14-16 for specialized attributes
* Waste Factor: Specialized classes minimize unused stats for their build type

**6. Build Distribution**

* 7 Physical-oriented classes (higher STR+DEX)
* 2 Magic-oriented classes (higher INT+FAI)
* 1 Hybrid class (balanced across all stats)

**7. Versatility Rankings**

* Wretch: 100% versatility (perfect balance)
* Confessor & Bandit: High versatility with slight specializations
* Specialized classes (Hero, Warrior, Astrologer, Prophet): Low versatility but optimal for focused builds

















### **💡 Recommendations**


**For New Players:**

* Start with Vagabond: High survivability (15 Vigor), good physical stats, forgiving for learning
* Alternative: Wretch: Maximum flexibility to experiment with all build types

**For Experienced Players:**

Choose Based on Your Build:

  * Pure Strength/Colossal Weapons → Hero
  * Dexterity/Fast Weapons → Warrior
  * Sorcery/Magic → Astrologer
  * Incantations/Faith → Prophet
  * Bleed/Status Effects → Bandit
  * Quality Weapons → Vagabond or Samura
  * Hybrid Spellcaster → Confessor

**Optimization Tips:**

  * Minimize Waste: Choose classes with low waste factors for your intended build (saves levels)
  * Plan for Vigor: Budget 25+ levels for reaching 40 Vigor regardless of class
  * Mind for Casters: Astrologer's 15 Mind provides significant early-game FP advantage
  * Respec Strategy: If planning multiple builds, Wretch provides the cleanest slate

Build Type Priority Matrix :

* High Survivability → Vagabond (15 Vigor)
* Maximum Damage (STR) → Hero (16 STR)
* Maximum Damage (DEX) → Warrior (16 DEX)
* Maximum Spell Power (INT) → Astrologer (16 INT)
* Maximum Spell Power (FAI) → Prophet (16 FAI)
* Most Flexible → Wretch (all 10s)
* Best Status Effects → Bandit (14 ARC)

**Final Recommendation:**

The "best" class depends entirely on your playstyle. For optimization-focused players, choose the specialist class matching your build to minimize wasted levels. For exploratory players, Wretch or Vagabond offer the most flexibility without significant disadvantages.




